In [1]:
WORKER_COUNT = 8

In [2]:
# Sync models
# gsutil -m rsync -r /mnt/seals/models/ gs://thesis-penguins/models/ 
# aws s3 sync /mnt/seals/models s3://thesisvids/penguins/models/

In [3]:
# whether to log each feature and sequence status
verbose = True

In [4]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 5000
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [5]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'
path_models = pwd + 'models/'

In [6]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [7]:
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed

Using TensorFlow backend.


# Helper functions

In [8]:
def get_results():
    """
    Load the results of all completed models into a dataframe and return sorted on val_acc
    """

    results = []

    for folder, subs, files in os.walk(path_models):
        for filename in files:
            if 'results.json' in filename:
                with open(os.path.abspath(os.path.join(folder, filename))) as f:
                    data = json.load(f)
                results.append(data)

    results = pd.DataFrame(results)        
    results.sort_values("fit_val_acc", inplace=True, ascending=False)
    return results

In [9]:
def remove_completed_experiments(experiments):
    """
    Helper function to remove the completed experiments from experiments dataframe
    """
    
    # get updated result set
    results = get_results()
    
    # merge results onto experiments
    experiments = pd.merge(experiments, results[['model_id','fit_val_acc']], left_on='model_id', right_on='model_id', how='left')
    experiments['done'] = (experiments['fit_val_acc']>0).astype(int)
    print("{}/{} experiments done".format(experiments[experiments['done'] == 1].shape[0], len(experiments)))
    
    # only keep experiments not done
    experiments = experiments[experiments['done'] == 0]

    # clean up
    del experiments['done']
    del experiments['fit_val_acc']
    
    # return experiments already done
    return experiments

In [10]:
def reassign_workers(experiments):
    """
    Helper function to reassign outstanding experiments to workers
    """
    
    
    # get updated result set
    results = get_results()
    
    # merge results onto experiments
    experiments = pd.merge(experiments, results[['model_id','fit_val_acc']], left_on='model_id', right_on='model_id', how='left')
    experiments['done'] = (experiments['fit_val_acc']>0).astype(int)
    
    # reassign workers
    global todo_idx
    todo_idx = 0
    
    def assign_not_done_to_worker(row):
        """ helper lambda"""
        global todo_idx
        if row['done'] == 0:
            todo_idx+=1
            return todo_idx % WORKER_COUNT
        else:
            return 0
    
    experiments['WORKER'] = experiments.apply(assign_not_done_to_worker, axis=1)
    
    # clean up
    del experiments['done']
    del experiments['fit_val_acc']
    
    return experiments

# Check status

In [ ]:
experiment_batch_name = 'experiment_batch_3'

In [ ]:
# reload experiments
experiments = pd.read_csv(pwd + "experiments/" + experiment_batch_name + ".csv")
experiments.shape

In [ ]:
# remove experiments already completed
experiments = remove_completed_experiments(experiments)
experiments.shape

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
experiments = reassign_workers(experiments)

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
experiments

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = pd.read_csv(pwd + "experiments/experiment_batch_3.csv")['model_id'].max() + 1
# read list of paths and find max there too to be double safe we don't overwrite model_ids
paths = os.listdir(path_models)
paths = [int(p) for p in paths]
model_id_start = max(model_id_start,max(paths) + 1)
print(model_id_start)

In [ ]:
experiments

In [ ]:
# add frame size column
model_id_start+=1
experiments['frame_size'] = None

new_experiment = {'WORKER': 5,
                     'architecture': 'c3dsmall',
                     'dropout': 0,
                     'layer_1_size': 0,
                     'layer_2_size': 0,
                     'layer_3_size': 0,
                     'model_id': model_id_start ,
                     'pooling': None,
                     'pretrained_model_name': None,
                     'sequence_length': 16,
                     'sequence_model': None,
                     'sequence_model_layers': None,
                     'frame_size': (112,112)}
experiments = experiments.append(new_experiment, ignore_index=True)
model_id_start+=1
new_experiment = {'WORKER': 4,
                     'architecture': 'c3d',
                     'dropout': 0,
                     'layer_1_size': 0,
                     'layer_2_size': 0,
                     'layer_3_size': 0,
                     'model_id': model_id_start,
                     'pooling': None,
                     'pretrained_model_name': None,
                     'sequence_length': 16,
                     'sequence_model': None,
                     'sequence_model_layers': None,
                     'frame_size':(112,112)}
experiments = experiments.append(new_experiment, ignore_index=True)
model_id +=1

In [ ]:
experiments

In [ ]:
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + ".csv", index=False)

# Create list of experiments to be run

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths, maybe try different convolution kernel sizes

* batch 3 = run even longer sequence lengths for 3 best models

* batch 4 = run trainable MLP and LRCNN on best performing frozen variants

* batch 5 = run trainable but initializing with best CNN weights

* batch 6 = run C3D models

* batch 7 = analyze effect of dropout and pooling with best model

# Batch 1

In [ ]:
experiment_batch_name = 'experiment_batch_1'

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = 0

In [ ]:
# init list of experiments
experiments = []

In [ ]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [ ]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [ ]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters
                        experiment['sequence_length'] = sequence_length

                        # add to list of experiments
                        experiments.append(experiment)

In [ ]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers
                                experiment['sequence_length'] = sequence_length

                                # add to list of experiments
                                experiments.append(experiment)

In [ ]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

### create model id column for this experiment batch
model_id_list = list(range(0,len(experiments)))
experiments['model_id'] = model_id_list

# assign to workers
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [ ]:
experiments.shape

In [ ]:
############################################
### remove invalid experiment configurations
############################################

# Just won't run experiments for those model_ids - not an error that model ids not congituous count from 0!

# delete video experiments with 0 neurons in a layer with nonzero neurons in later layers
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_2_size'] > 0))]
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_3_size'] > 0))]
experiments = experiments[~((experiments['layer_2_size'] == 0) & (experiments['layer_3_size'] > 0))]

# delete video experiments where convolution_kernel_size > sequence_length (convolution_kernel_size defaults to 3 and not set in this batch)
experiments = experiments[~((experiments['sequence_model'] == 'Convolution1D') & (experiments['sequence_length']<=3))]

In [ ]:
# delete LRCNN_frozen experiments with layer_1_size == 0
experiments = experiments[~((experiments['architecture'] == 'video_LRCNN_frozen') & (experiments['layer_1_size']==0))]

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [ ]:
print(experiments.shape)
experiments.tail().T

In [ ]:
# # upload to s3
# response = os.system("aws s3 cp " + pwd + "experiments/" + experiment_batch_name + '.csv s3://thesisvids/penguins/' + experiment_batch_name + '.csv')
# if response == 0:
#     print("upload success")
# else:
#     print("upload error")

In [ ]:
# # # upload to GCP
# response = os.system("gsutil cp " + pwd + "experiments/" + experiment_batch_name + '.csv gs://thesis-penguins/' + experiment_batch_name + '.csv')
# if response == 0:
#     print("upload success")
# else:
#     print("upload error")

# Batch 2

In [ ]:
# reload experiments
experiments = pd.read_csv(pwd + "experiments/experiment_batch_1.csv")
experiments.shape

In [ ]:
# define experiment batch name
experiment_batch_name = 'experiment_batch_2'

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = pd.read_csv(pwd + "experiments/experiment_batch_1.csv")['model_id'].max() + 1
# read list of paths and find max there too to be double safe we don't overwrite model_ids
paths = os.listdir(path_models)
paths = [int(p) for p in paths]
model_id_start = max(model_id_start,max(paths) + 1)
print(model_id_start)

In [ ]:
# remove experiments already completed
experiments = remove_completed_experiments(experiments)
experiments.shape

In [ ]:
# get top 20 results from previous batch
results = get_results()

In [ ]:
# we'll create experiments with same parameters as best 20 so far but now with longer sequence lengths
results_top20 = results[['architecture','layer_1_size','layer_2_size', 'layer_3_size','dropout','pretrained_model_name','pooling','sequence_length','sequence_model','sequence_model_layers']].head(20)
sequence_lengths = [15, 20]

In [ ]:
# init list for extra experiments to append to outstanding experiments
experiments_next = []

# init model_id
model_id = model_id_start

for sequence_length in sequence_lengths:
    for result in results_top20.values:
        
        # convert result params to dict to create experiment from
        result_dict = dict(zip(results_top20.columns, result))

        experiment = {}                    
        experiment['architecture'] = 'video_lrcnn_frozen'
        experiment['sequence_model'] = result_dict['sequence_model']
        experiment['sequence_model_layers'] = result_dict['sequence_model_layers']
        experiment['sequence_length'] = sequence_length
        experiment['pretrained_model_name'] = result_dict['pretrained_model_name']
        experiment['layer_1_size'] = result_dict['layer_1_size']
        experiment['layer_2_size'] = result_dict['layer_2_size']
        experiment['layer_3_size'] = result_dict['layer_3_size']
        experiment['dropout'] = result_dict['dropout']
        experiment['pooling'] = 'max' # always outperforms avg
        
        experiment['model_id'] = model_id
        
        # assign to worker
        experiment['WORKER'] = model_id % WORKER_COUNT

        # increment unique model id
        model_id +=1
        
        # add to list of additional experiments
        experiments_next.append(experiment)

In [ ]:
# convert to dataframe
experiments_next = pd.DataFrame(experiments_next)
experiments_next.shape

In [ ]:
# merge new experiments onto outstanding experiments
experiments = pd.concat([experiments, experiments_next], axis = 0)
experiments.shape

In [ ]:
experiments.tail(100)

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
experiments = reassign_workers(experiments)

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [ ]:
# # upload to s3
# response = os.system("aws s3 cp " + pwd + "experiments/" + experiment_batch_name + '.csv s3://thesisvids/penguins/' + experiment_batch_name + '.csv')
# if response == 0:
#     print("upload success")
# else:
#     print("upload error")

In [ ]:
# # upload to GCP
response = os.system("gsutil cp " + pwd + "experiments/" + experiment_batch_name + '.csv gs://thesis-penguins/' + experiment_batch_name + '.csv')
if response == 0:
    print("upload success")
else:
    print("upload error")

# Batch 3

In [ ]:
# reload experiments
experiments = pd.read_csv(pwd + "experiments/experiment_batch_2.csv")
experiments.shape

In [ ]:
# define experiment batch name
experiment_batch_name = 'experiment_batch_3'

In [ ]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id_start = pd.read_csv(pwd + "experiments/experiment_batch_2.csv")['model_id'].max() + 1
# read list of paths and find max there too to be double safe we don't overwrite model_ids
paths = os.listdir(path_models)
paths = [int(p) for p in paths]
model_id_start = max(model_id_start,max(paths) + 1)
print(model_id_start)

In [ ]:
# remove experiments already completed
experiments = remove_completed_experiments(experiments)
experiments.shape

In [ ]:
# get top 20 results from previous batch
results = get_results()

In [ ]:
# we'll create experiments with same parameters as best 20 so far but now with longer sequence lengths
results_top = results[['architecture','layer_1_size','layer_2_size', 'layer_3_size','dropout','pretrained_model_name','pooling','sequence_length','sequence_model','sequence_model_layers']].head(20)
sequence_lengths = [25, 30, 35, 40, 45, 50, 75, 100]

In [ ]:
results_top = results_top[results_top['sequence_length'] == 20].head(3)

In [ ]:
results_top.head().T

In [ ]:
# init list for extra experiments to append to outstanding experiments
experiments_next = []

# init model_id
model_id = model_id_start

for sequence_length in sequence_lengths:
    for result in results_top.values:
        
        # convert result params to dict to create experiment from
        result_dict = dict(zip(results_top.columns, result))

        experiment = {}                    
        experiment['architecture'] = 'video_lrcnn_frozen'
        experiment['sequence_model'] = result_dict['sequence_model']
        experiment['sequence_model_layers'] = result_dict['sequence_model_layers']
        experiment['sequence_length'] = sequence_length
        experiment['pretrained_model_name'] = result_dict['pretrained_model_name']
        experiment['layer_1_size'] = result_dict['layer_1_size']
        experiment['layer_2_size'] = result_dict['layer_2_size']
        experiment['layer_3_size'] = result_dict['layer_3_size']
        experiment['dropout'] = result_dict['dropout']
        experiment['pooling'] = 'max' # always outperforms avg
        
        experiment['model_id'] = model_id
        
        # assign to worker
        experiment['WORKER'] = model_id % WORKER_COUNT

        # increment unique model id
        model_id +=1
        
        # add to list of additional experiments
        experiments_next.append(experiment)

In [ ]:
# convert to dataframe
experiments_next = pd.DataFrame(experiments_next)
experiments_next.shape

In [ ]:
# merge new experiments onto outstanding experiments
experiments = pd.concat([experiments, experiments_next], axis = 0)
experiments.shape

In [ ]:
experiments.tail(100)

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
experiments = remove_completed_experiments(experiments)

In [ ]:
experiments = reassign_workers(experiments)

In [ ]:
experiments

In [ ]:
experiments['WORKER'].value_counts()

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

In [ ]:
# # upload to s3
# response = os.system("aws s3 cp " + pwd + "experiments/" + experiment_batch_name + '.csv s3://thesisvids/penguins/' + experiment_batch_name + '.csv')
# if response == 0:
#     print("upload success")
# else:
#     print("upload error")

In [ ]:
# # upload to GCP
response = os.system("gsutil cp " + pwd + "experiments/" + experiment_batch_name + '.csv gs://thesis-penguins/' + experiment_batch_name + '.csv')
if response == 0:
    print("upload success")
else:
    print("upload error")

# Batch 4

# Batch 5

# Batch 6

# Analyze results 

## load results.json for all models into dataframe

In [11]:
results = get_results()

In [12]:
results.groupby("architecture").agg('max')['fit_val_acc']

architecture
c3dsmall              0.919419
image_mlp_frozen      0.946495
video_lrcnn_frozen    0.958697
video_mlp_concat      0.948735
Name: fit_val_acc, dtype: float64

In [13]:
results.head(20).T

,1181,1457,1129,2802,198,3261,211,1993,1308,1843,201,2852,1908,2452,1699,1219,2571,2794,2741,884
architecture,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen
batch_size,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32
convolution_kernel_size,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
data_total_rows_test,3120,3120,3130,3120,3130,3125,3110,3130,3120,3090,3115,3130,3120,3120,3120,3125,3125,3130,3130,3105
data_total_rows_train,60179,60179,60399,60179,60399,60289,59959,60399,60179,59519,60069,60399,60179,60179,60179,60289,60289,60399,60399,59849
data_total_rows_valid,6378,6378,6398,6378,6398,6388,6358,6398,6378,6318,6368,6398,6378,6378,6378,6388,6388,6398,6398,6348
dropout,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
fit_best_round,3,2,2,3,3,3,2,3,2,2,2,3,2,3,3,3,2,3,3,1
fit_dt_test_duration_seconds,6,3,15,4,31,4,3,26,3,7,3,12,5,4,4,2,4,27,15,4
fit_dt_test_end,2019-01-25 09:32:01,2019-01-25 09:49:36,2019-01-24 07:02:41,2019-01-25 10:02:33,2019-01-23 03:34:38,2019-01-25 08:07:33,2019-01-25 15:21:24,2019-01-22 22:58:13,2019-01-25 12:50:18,2019-01-25 18:40:00,2019-01-25 15:22:09,2019-01-24 03:25:23,2019-01-25 11:12:07,2019-01-25 11:44:58,2019-01-25 13:10:15,2019-01-25 07:49:22,2019-01-25 08:48:58,2019-01-23 00:18:48,2019-01-24 08:58:42,2019-01-25 15:23:26


> top 20 models all `video_lrcnn_frozen` with `vgg16` and `LSTM`, `SimpleRNN`, `GRU` sequence models and sequence_length = 10 ... we'll re-run these with sequence length = 15 and 20 for batch 2

In [ ]:
# results[results['model_id'].isin([362, 550, 162, 133, 3115, 3125])].T

## results per architecture

In [14]:
results[results['architecture'] == 'video_mlp_concat'].head(5).T

,1131,1255,1792,3033,3296
architecture,video_mlp_concat,video_mlp_concat,video_mlp_concat,video_mlp_concat,video_mlp_concat
batch_size,32,32,32,32,32
convolution_kernel_size,3,3,3,3,3
data_total_rows_test,3137,3135,3137,3137,3137
data_total_rows_train,60553,60509,60553,60553,60553
data_total_rows_valid,6412,6408,6412,6412,6412
dropout,0.2,0.2,0.2,0.2,0.2
fit_best_round,3,3,2,1,2
fit_dt_test_duration_seconds,1,0,1,0,0
fit_dt_test_end,2019-01-20 21:15:03,2019-01-20 23:33:37,2019-01-20 21:37:34,2019-01-20 20:46:20,2019-01-20 21:17:15


In [15]:
results[results['architecture'] == 'image_mlp_frozen'].head(5).T

,2586,2551,1260,3086,2326
architecture,image_mlp_frozen,image_mlp_frozen,image_mlp_frozen,image_mlp_frozen,image_mlp_frozen
batch_size,32,32,32,32,32
convolution_kernel_size,3,3,3,3,3
data_total_rows_test,3139,3139,3139,3139,3139
data_total_rows_train,60597,60597,60597,60597,60597
data_total_rows_valid,6416,6416,6416,6416,6416
dropout,0.2,0.2,0.2,0.2,0.2
fit_best_round,2,2,2,3,2
fit_dt_test_duration_seconds,2,1,1,1,1
fit_dt_test_end,2019-01-20 14:25:08,2019-01-20 14:01:02,2019-01-20 14:07:05,2019-01-20 13:54:52,2019-01-20 14:13:38


In [16]:
results[results['architecture'] == 'video_lrcnn_frozen'].head(5).T

,1181,1457,1129,2802,198
architecture,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen,video_lrcnn_frozen
batch_size,32,32,32,32,32
convolution_kernel_size,3,3,3,3,3
data_total_rows_test,3120,3120,3130,3120,3130
data_total_rows_train,60179,60179,60399,60179,60399
data_total_rows_valid,6378,6378,6398,6378,6398
dropout,0.2,0.2,0.2,0.2,0.2
fit_best_round,3,2,2,3,3
fit_dt_test_duration_seconds,6,3,15,4,31
fit_dt_test_end,2019-01-25 09:32:01,2019-01-25 09:49:36,2019-01-24 07:02:41,2019-01-25 10:02:33,2019-01-23 03:34:38


## analyze best combinations

In [17]:
results.groupby("sequence_model").agg('max')['fit_val_acc']

sequence_model
Convolution1D    0.936084
GRU              0.956125
LSTM             0.958697
SimpleRNN        0.958317
Name: fit_val_acc, dtype: float64

In [18]:
results.groupby("sequence_length").agg('max')['fit_train_acc']

sequence_length
1.0      0.976284
3.0      0.977838
5.0      0.978121
10.0     0.984162
15.0     0.981510
16.0     0.976100
20.0     0.982184
25.0     0.973766
30.0     0.986479
35.0     0.988199
40.0     0.975228
45.0     0.974495
50.0     0.973653
75.0     0.978139
100.0    0.989561
Name: fit_train_acc, dtype: float64

In [ ]:
results.groupby("sequence_length").agg('max')['fit_val_acc']

In [ ]:
results[results['architecture']=='video_lrcnn_frozen'].groupby("sequence_length").agg('max')['fit_val_acc'].plot()

In [ ]:
grp = results.groupby(["sequence_model","sequence_length","sequence_model_layers"]).agg("max")['fit_val_acc'].unstack().sort_values("sequence_length")
grp

In [ ]:
grp.reset_index(level=1, drop=True, inplace=True)

In [ ]:
grp.columns=['one','two']

In [ ]:
grp['gap'] = (grp['two'] - grp['one'] )/ grp['two']

In [ ]:
grp

> two sequence levels better than 1

In [ ]:
grp = results.groupby(["sequence_model","sequence_length"]).agg("max")['fit_val_acc'].unstack()
grp

In [ ]:
# sort cols
grpcols = list(grp.columns)
grpcols.sort()
grp = grp[grpcols]
# plot
plt = grp.plot(kind='bar', figsize=(10,5))
x1,x2,y1,y2 = plt.axis()
plt.axis((x1,x2,0.85,1))

In [ ]:
results.sort_values("fit_val_acc",ascending=False).head(30).plot(x='model_param_count',y='fit_val_acc', kind='scatter')

In [ ]:
results.sort_values('fit_dt_test_end',ascending=False).head().T

# re-allocate experiments to workers

In [ ]:
# remove experiments already done
experiments = remove_completed_experiments(experiments)

## re-assign workers on the experiments that are not complete

In [ ]:
# reassign outstanding experiments to workers
experiments = reassign_workers(experiments)

In [ ]:
##################################
### output experiment batch to CSV
##################################
print(experiment_batch_name)
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

# Debug experiment worker

In [ ]:
from deepvideoclassification.architectures import Architecture

In [ ]:
WORKER_ID = 0
GPU_ID = 0,1,2,3
experiment_batch_name = 'experiment_batch_3'

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=str(GPU_ID)

In [ ]:
# setup logging
# separate log file for each worker
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs_" + str(WORKER_ID))),
        logging.StreamHandler()
    ])
# init logger - will pass this to our architecture
logger = logging.getLogger()

logger.info("Start worker {} (GPU={}) processing {}".format(WORKER_ID, GPU_ID, experiment_batch_name))

In [ ]:
# load list of experiments
experiments = pd.read_csv(pwd + "experiments/" + experiment_batch_name + '.csv')

In [ ]:
experiments

In [ ]:
# for row in experiments.values:
debug_model_id = 5323

row = list(experiments[experiments['model_id'] == debug_model_id].values[0])

# get experiment params from dataframe row
experiment = dict(zip(experiments.columns, row))

In [ ]:
print(str(experiment["model_id"]) + "   " + "X"*60)
#logging.info("Begin experiment for model_id={} on GPU:{} ".format(experiment['model_id'], os.environ["CUDA_VISIBLE_DEVICES"]))
print(experiment)

architecture = Architecture(model_id = experiment['model_id'], 
                            architecture = experiment['architecture'], 
                            sequence_length = experiment['sequence_length'], 
                            pretrained_model_name = experiment['pretrained_model_name'],
                            pooling = experiment['pooling'],
                            sequence_model = experiment['sequence_model'],
                            sequence_model_layers = experiment['sequence_model_layers'],
                            layer_1_size = experiment['layer_1_size'],
                            layer_2_size = experiment['layer_2_size'],
                            layer_3_size = experiment['layer_3_size'],
                            dropout = experiment['dropout'],
                            verbose=True,
                            batch_size=32)

In [ ]:
architecture.train_model()